In [1]:
import os
import torch
import sys
sys.path.append('../../lib/exlib/src')

import sys
sys.path.append('../../src')
import sop

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sop.utils.seed_all(42)

# config
exp_config = sop.ImageNetConfig()
val_config = exp_config.get_config('val_sm')
val_config['evaluation']['batch_size'] = 16
val_config

/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may hav

{'dataset': {'name': 'imagenet_s',
  'root': '/shared_data0/weiqiuy/datasets/imagenet'},
 'training': {'batch_size': 16,
  'num_epochs': 20,
  'mask_batch_size': 64,
  'optimizer': {'name': 'adamw', 'lr': 5e-06, 'weight_decay': 0.01}},
 'evaluation': {'split': 'val', 'num_data': 1, 'batch_size': 16},
 'model': {'type': 'vit',
  'base': 'google/vit-base-patch16-224',
  'sop': '/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/best',
  'num_classes': 1000}}

In [2]:
# model
backbone_model, processor, backbone_config = sop.utils.imagenet_utils.get_model(val_config['model']['type'],
                                                                 backbone_model_name=val_config['model']['base'],
                                                                 backbone_processor_name=val_config['model']['base'],
                                                                )
backbone_model = backbone_model.to(device)

# get wrapped original model
from sop.utils.imagenet_utils import WrappedModel

original_model = WrappedModel(backbone_model, output_type='logits')
original_model = original_model.to(device)
original_model.eval();

# config
from exlib.modules.sop import SOPConfig, get_chained_attr

config = SOPConfig(os.path.join(val_config['model']['sop'], 'config.json'))

# config.group_sel_scale = 0.05

config.__dict__.update(backbone_config.__dict__)
config.num_labels = len(backbone_config.id2label)

# get sop model
from sop.utils.imagenet_utils import get_model, get_wrapped_models

wrapped_backbone_model, class_weights, projection_layer = get_wrapped_models(
    backbone_model,
    config
)
wrapped_backbone_model = wrapped_backbone_model.to(device)
wrapped_backbone_model.eval();
class_weights = class_weights.to(device)
projection_layer = projection_layer.to(device)

# sop
from exlib.modules.sop import SOPImageCls4

model = SOPImageCls4(config, wrapped_backbone_model, 
                     class_weights=class_weights, 
                     projection_layer=projection_layer)
state_dict = torch.load(os.path.join(val_config['model']['sop'], 
                                     'checkpoint.pth'))
print('Loaded step', state_dict['step'])
model.load_state_dict(state_dict['model'], strict=False)
model = model.to(device)
model.eval();

projection layer is not frozen
projection layer is not frozen
Loaded step 40100


In [3]:
val_config['model']['sop']

'/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/best'

In [4]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

In [5]:
from sop.metrics import get_ins_del_perc

In [10]:
method = 'shap_20'

# get results example

In [8]:
# results_ins = {}
# for method in methods:
results_ins = get_ins_del_perc(val_config, original_model, backbone_model, model, processor,
                     method, debug=True)

shap_20


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

0.8887572074515773


In [11]:
results_del = get_ins_del_perc(val_config, original_model, backbone_model, model, processor,
                     method, debug=True, deletion=True)

shap_20


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

0.5132429386690466


# Results

In [ ]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

In [26]:
for method in methods:
    data = torch.load(f'/shared_data0/weiqiuy/sop/results/ins_del/imagenet_s/{method}.pt')
    print(len(data['ins']['scores_curve_perc']))
    print(len(data['del']['scores_curve_perc']))
    
    print(method, 'ins', np.mean(data['ins']['scores_curve_perc']), 
          'del', np.mean(data['del']['scores_curve_perc']))

16
16
shap_20 ins 0.8887572074515773 del 0.5132429386690466


FileNotFoundError: [Errno 2] No such file or directory: '/shared_data0/weiqiuy/sop/results/ins_del/imagenet_s/rise_20.pt'

In [18]:
data = torch.load('/shared_data0/weiqiuy/sop/results/ins_del/imagenet_s/lime_20.pt')

In [19]:
data.keys(), data['ins'].keys(), data['ins']['scores_curve_perc'][:10]

(dict_keys(['ins', 'del']),
 dict_keys(['scores_mean', 'scores_curve', 'scores_curve_perc']),
 [0.30941758659424384,
  0.5342400458198,
  0.7519785941593089,
  0.5726131650617945,
  0.982614019327433,
  0.893907126682252,
  0.8472534268134003,
  0.9473662766869041,
  0.8753398389306111,
  0.9279017636587231])

In [22]:
import numpy as np

np.mean(data['ins']['scores_curve_perc'])

0.8209578853120209

In [ ]:
for k in results_ins['scores_curve_perc']:
    print(k, np.mean(results_ins['scores_curve_perc'][k]))

In [ ]:
results_del = []
for explainer_name in methods:
    results_del[method] = get_ins_del_perc(val_config, original_model, backbone_model, model, processor,
                     explainer_name, debug=True, deletion=True)

In [ ]:
for k in results_del['scores_curve_perc']:
    print(k, np.mean(results_del['scores_curve_perc'][k]))

# Accuracy

In [9]:
def get_acc(explainer_name, suffix='', debug=False):
    method = explainer_name.split('_')[0]
    if explainer_name == 'bagnet':
        ATTR_VAL_DATA_DIR = None
    else:
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name.replace("-s", "")}_1_pred{suffix}/val'

    explainer = sop.utils.get_explainer(original_model, backbone_model, method.split('_')[0], device)

    val_dataset, val_dataloader = sop.utils.get_dataset(val_config['dataset']['name'], 
                                          split=val_config['evaluation']['split'], 
                                          num_data=val_config['evaluation']['num_data'],
                                          batch_size=val_config['evaluation']['batch_size'],
                                                        attr_dir=ATTR_VAL_DATA_DIR,
                                          processor=processor, debug=debug)

    corrects = []
    for bi, batch in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        if debug and bi >= 3:
            break
        # if bi != len(val_dataloader) - 1:
        #     continue
        if len(batch) == 5:
            inputs, labels, segs, attrs, idxs = batch
        else:
            inputs, labels, segs, idxs = batch
            attrs = None
        inputs, labels, segs = inputs.to(device), labels.to(device), segs.to(device)

        inputs_norm = inputs
        # inputs_norm = (inputs_norm + 1) / 2
        inputs_norm = explainer.preprocess(inputs_norm)
        print('inputs_norm', inputs_norm.shape)
        # inputs_norm = (inputs_norm + 1) / 2
        outputs = explainer.model(inputs_norm)
        probs = outputs.softmax(-1)
        preds = probs.argmax(-1)
        print(preds, labels, preds == labels)
        corrects.extend((preds == labels).cpu().tolist())
    print(sum(corrects) / len(corrects))

In [11]:
get_acc('bcos', debug=True)

Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/100 [00:00<?, ?it/s]

inputs_norm torch.Size([1, 6, 224, 224])
tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
inputs_norm torch.Size([1, 6, 224, 224])
tensor([1], device='cuda:0') tensor([1], device='cuda:0') tensor([True], device='cuda:0')
inputs_norm torch.Size([1, 6, 224, 224])
tensor([2], device='cuda:0') tensor([2], device='cuda:0') tensor([True], device='cuda:0')
1.0
